<a href="https://colab.research.google.com/github/AtomGen-2/recommendor/blob/main/RecommendorV1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install turicreate

     |████████████████████████████████| 92.0 MB 5.4 kB/s 
     |████████████████████████████████| 3.6 MB 47.3 MB/s 
     |████████████████████████████████| 86.4 MB 60 kB/s 
     |████████████████████████████████| 3.5 MB 16.7 MB/s 
     |████████████████████████████████| 322 kB 58.8 MB/s 
     |████████████████████████████████| 18.3 MB 75 kB/s 
     |████████████████████████████████| 449 kB 45.3 MB/s 
     |████████████████████████████████| 50 kB 4.1 MB/s 
     |████████████████████████████████| 2.9 MB 45.3 MB/s 
     |████████████████████████████████| 20.1 MB 2.1 MB/s 
     |████████████████████████████████| 3.8 MB 39.2 MB/s 
  Created wheel for prettytable: filename=prettytable-0.7.2-py3-none-any.whl size=13714 sha256=b03c2788d29aba3738b48a3337289a1725199f2210fa966722d78e12d46f5abf
  Stored in directory: /root/.cache/pip/wheels/b2/7f/f6/f180315b584f00445045ff1699b550fa895d09471337ce21c6
  Created wheel for resampy: filename=resampy-0.2.1-py3-none-any.whl size=320860 sha256=b8937dc7dd3

In [5]:
pip install scripts

In [6]:
import pandas as pd
import numpy as np
import time
import turicreate as tc
from sklearn.model_selection import train_test_split

import sys
sys.path.append("..")
# import scripts.data_layer as data_layer
import scripts

In [10]:
import requests
url = 'https://raw.githubusercontent.com/moorissa/medium/master/items-recommender/data/recommend_1.csv'
res = requests.get(url, allow_redirects=True)
with open('recommend_1.csv','wb') as file:
    file.write(res.content)
customers = pd.read_csv('recommend_1.csv')

In [11]:
print(customers.shape)
customers.head()

(1000, 1)


,customerId
0,1553
1,20400
2,19750
3,6334
4,27773


In [14]:
import requests
url = 'https://raw.githubusercontent.com/moorissa/medium/master/items-recommender/data/trx_data.csv'
res = requests.get(url, allow_redirects=True)
with open('trx_data.csv','wb') as file:
    file.write(res.content)
transactions = pd.read_csv('trx_data.csv')

In [15]:
print(transactions.shape)
transactions.head()

(62483, 2)


,customerId,products
0,0,20
1,1,2|2|23|68|68|111|29|86|107|152
2,2,111|107|29|11|11|11|33|23
3,3,164|227
4,5,2|2


In [16]:
# example 1: split product items
transactions['products'] = transactions['products'].apply(lambda x: [int(i) for i in x.split('|')])
transactions.head(2).set_index('customerId')['products'].apply(pd.Series).reset_index()

,customerId,0,1,2,3,4,5,6,7,8,9
0,0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2.0,2.0,23.0,68.0,68.0,111.0,29.0,86.0,107.0,152.0


In [17]:
# example 2: organize a given table into a dataframe with customerId, single productId, and purchase count
pd.melt(transactions.head(2).set_index('customerId')['products'].apply(pd.Series).reset_index(), 
             id_vars=['customerId'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['customerId', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})

,customerId,productId,purchase_count
0,0,20.0,1
1,1,2.0,2
2,1,23.0,1
3,1,29.0,1
4,1,68.0,2
5,1,86.0,1
6,1,107.0,1
7,1,111.0,1
8,1,152.0,1


In [18]:
data = pd.melt(transactions.set_index('customerId')['products'].apply(pd.Series).reset_index(), 
             id_vars=['customerId'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['customerId', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})
data['productId'] = data['productId'].astype(np.int64)

In [19]:
print(data.shape)
data.head

(133585, 3)


<bound method NDFrame.head of         customerId  productId  purchase_count
0                0          1               2
1                0         13               1
2                0         19               3
3                0         20               1
4                0         31               2
...            ...        ...             ...
133580       28596        211               3
133581       28596        255               1
133582       28598        212               1
133583       28604        282               1
133584       28605         92               1

[133585 rows x 3 columns]>

In [20]:
def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy['purchase_dummy'] = 1
    return data_dummy
data_dummy = create_data_dummy(data)

In [21]:
df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')

In [22]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())

In [23]:
# create a table for input to the modeling

d = df_matrix_norm.reset_index()
d.index.names = ['scaled_purchase_freq']
data_norm = pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()
print(data_norm.shape)
data_norm.head()


(133585, 3)


,customerId,productId,scaled_purchase_freq
9,9,0,0.133333
25,25,0,0.133333
32,33,0,0.133333
35,36,0,0.133333
43,44,0,0.133333


In [24]:
def normalize_data(data):
    df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')
    df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
    d = df_matrix_norm.reset_index()
    d.index.names = ['scaled_purchase_freq']
    return pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()

In [25]:
def split_data(data):
    '''
    Splits dataset into training and test set.
    
    Args:
        data (pandas.DataFrame)
        
    Returns
        train_data (tc.SFrame)
        test_data (tc.SFrame)
    '''
    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

In [26]:
train_data, test_data = split_data(data)
train_data_dummy, test_data_dummy = split_data(data_dummy)
train_data_norm, test_data_norm = split_data(data_norm)